# Parallelized binary dilation ("droplet-finding")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.morphology import dilation
from trackpy import bandpass
from skimage.io import imread

img = imread('data/from-andi.png', as_gray=True)

In [ ]:
plt.imshow(img)

In [ ]:
result_numpy = dilation(bandpass(img, 1, 15) > 0.2)
plt.imshow(result_numpy)

In [ ]:
from dask.distributed import Client, progress
import dask.array

client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

In [ ]:
dask_img = dask.array.from_array(img, chunks=(256, 256))

In [ ]:
lazy_result = ((dask_img
    .map_overlap(
        lambda block: bandpass(block, 1, 15), depth=15) > 0.2)
    .map_overlap(
        lambda block: dilation(block), depth=3))

In [ ]:
result = lazy_result.compute()

In [ ]:
plt.imshow(result)